In [ ]:
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

# Data beans common themes by item summarization using RAG

This notebook shows how to implement a RAG procedure inside BigQuery to extract common review themes by product name, it performs the following steps:
- Create LLM and embeddings models (`gemini-pro` and `gecko`)
- Embbed the `review_text` column of the `customer_review` table
- Create an index on the embeddings for faster retrieval
- Wraps inside a BigQuery procedure a RAG implementation

In [ ]:
PROJECT_ID = "${project_id}"
REGION = "us"
DATASET_ID = "${bigquery_data_beans_curated_dataset}"
CONNECTION_NAME = "vertex-ai"

In [ ]:
%%bigquery
CREATE SCHEMA IF NOT EXISTS `${project_id}.${bigquery_data_beans_curated_dataset}_local` OPTIONS(location = 'US');

CREATE OR REPLACE MODEL `${project_id}.${bigquery_data_beans_curated_dataset}_local.emb_model`
 REMOTE WITH CONNECTION `us.vertex-ai`
 OPTIONS(ENDPOINT = 'textembedding-gecko@latest');

CREATE OR REPLACE MODEL `${project_id}.${bigquery_data_beans_curated_dataset}_local.llm_model`
 REMOTE WITH CONNECTION `us.vertex-ai`
 OPTIONS(ENDPOINT = 'gemini-pro');

Create table with embeddings

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `${project_id}.${bigquery_data_beans_curated_dataset}_local.customer_review_embedded`
as
SELECT *
FROM
  ML.GENERATE_TEXT_EMBEDDING(
    MODEL `${project_id}.${bigquery_data_beans_curated_dataset}_local.emb_model`,
    (select review_text as content from `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review` )
);

Create vector index on the embeddings table

In [ ]:
%%bigquery
CREATE OR REPLACE VECTOR INDEX `${project_id}.${bigquery_data_beans_curated_dataset}_local.reviews_index`
ON `${project_id}.${bigquery_data_beans_curated_dataset}_local.customer_review_embedded`(text_embedding)
OPTIONS(distance_type='COSINE', index_type='IVF');

Create a procedure to implement a RAG pipeline

In [ ]:
%%bigquery
CREATE OR REPLACE PROCEDURE ${bigquery_data_beans_curated_dataset}_local.common_themes_by_menu_items(menu_item STRING, OUT themes STRING)
BEGIN
SELECT
  ml_generate_text_llm_result AS generated
FROM
  ML.GENERATE_TEXT( MODEL `${project_id}.${bigquery_data_beans_curated_dataset}_local.llm_model`,
    (
    SELECT
      CONCAT('Extract common themes from the following reviews: ', STRING_AGG(FORMAT("review text: %s", base.content), ',\n'),
      '. Reply in JSON format with this format: {"item_name": ITEM_NAME_HERE, "common_themes": [COMMON_THEMES_LIST_HERE]]}') AS prompt,
    FROM
      VECTOR_SEARCH( TABLE `${project_id}.${bigquery_data_beans_curated_dataset}_local.customer_review_embedded`,
        'text_embedding',
        (
        SELECT
          text_embedding,
          content AS query
        FROM
          ML.GENERATE_TEXT_EMBEDDING( MODEL `${project_id}.${bigquery_data_beans_curated_dataset}_local.emb_model`,
            (
            SELECT
              CAST(menu_item AS STRING) AS content)) ),
        top_k => 5) ),
    STRUCT(0.4 AS temperature,
      300 AS max_output_tokens,
      0.5 AS top_p,
      5 AS top_k,
      TRUE AS flatten_json_output));
END

Executes the procedure

In [ ]:
%%bigquery
DECLARE themes STRING;
CALL ${bigquery_data_beans_curated_dataset}_local.common_themes_by_menu_items('capuccino',themes);